In [ ]:
def Eulirian_Path (adjacency_dic):
    """ Input : Dic with str as keys (a kmers' prefix [kmer[:-1]) and LIST with str as values (same kmers' suffix [kmer[1:]). 
                The key and its value(s) are adjacent nodes/vertices that together form a kmer. See ouput of DeBruijn().
                
        Output: list with int or str as elements. Elements in list indicate nodes and must be as many as the input's dict values() 
                + 1 element that is inserted at the start to indicate a key (start) node in contrast to the rest which are all dict values.
                                                                                                                        
                From a graph (input), checks whether a Eulirian Path exists. If so, a Eulirian cycle is first created by joining the
                edges (extreme nodes and NOT graph edges) of the graph. Then, the Eulirian Path is found by rotating the final Eulirian Cycle accordingly
    
                                                                                                                        """
    import random
    from collections import deque
    
    N_of_nodes   = sum (list(map(lambda x: len(x), adjacency_dic.values())))             # dict values are lists, thus measure the elements inside them (nodes)           
    first_key,last_key = Eulirian_Edges (adjacency_dic)                                  # These should be the final first and last nodes
    adjacency_dic = Path_With_Joined_Edges (adjacency_dic,first_key,last_key)            # Create an artificial cycle by joining first and last node
    cycle        = deque([first_key, adjacency_dic[first_key].pop()])                    # first dict value element is accessed via first key, first edge is used.
    exit         = 0                                                                     # distance of first exit node after a cycle, from the end of the next cycle 
    start_key    = cycle[0]                                                              # the only element of cycle that represents key instead of value. Always in start.
      
    while len(cycle) != N_of_nodes +1 :                                                  #+1 because Eulirian cycle's length = a start key node in start + dict values' elements 
        current_node = cycle[-1]                                                         

        if len(adjacency_dic[current_node]) > 1 and exit == 0 :                          # at least two elements in dic value (list) and no exit node found yet. exit == 0 to prevent from changing exit_nodes when new ones are entered
            exit_node = cycle[-1]                                                        ## only the first exit node is needed in every new cycle for wayout
        
        if adjacency_dic[current_node] != [] :                                           # at least one element in dic value(list)
            current_node = adjacency_dic[current_node].pop()                             
            cycle.append(current_node)
            try:
                exit_node
                exit +=1                                                                # first exit node's distance from end of cycle increases with new elements
            except:                                                                     # without an exit node present, don't measure distance
                pass      

        else :                                                                          #dead-end, no value left for this node
            cycle.remove(start_key)                                                     #remove the start key from cycle. This will also facilitate rotating exit node to the start instead of end of cycle
            cycle.rotate(exit)                                                          #move exit node to the start of the cycle and maintain elements' sequence
            cycle.append (cycle[0])                                                     #append the exit node at the end of the cycle to complete a cycle and continue from there
            start_key = cycle[0]                                                        #save the start key for the next dead-end
            exit = 0                                                                    #exit=0 so that we can find the first exit node and save it                                                               
    
    while cycle[-1] != last_key:                                                        #as long as the last key is not at last position
        cycle.rotate(-cycle.index(first_key))                                           #move all the nodes to the left until a first key occurence is at first place
  
    return cycle 


In [ ]:
def Eulirian_Edges (dic):
    """Input : Dic with str as keys (a kmers' prefix [kmer[:-1]) and LIST with str as values (same kmers' suffix [kmer[1:]). 
                The key and its value(s) are adjacent nodes/vertices that together form a kmer. See ouput of DeBruijn()
       Output: list with int or str containing the first and last node of a path which has not been unraveled yet.
       
       Check whether a Eulirian path exists (2 unbalanced nodes and a graph which is connected)
       Note that an 'edge' here indicates that these nodes are at the extremes of the path. Thus, it doesn't indicate graph edges"""
    
    
    outdegrees_indegrees_sum = dict.fromkeys(dic, 0)
    more_afferents = []                                                   # nodes with more indegrees than outdegrees
    more_efferents = []                                                   # nodes with more outdegrees than outdegrees

    for key, values in dic.items():                              
        outdegrees_indegrees_sum[key] += len(values)                      # add N of outdegrees for each key-node
        
    for key,values in dic.items():                               
        for node in values:
            try :
                outdegrees_indegrees_sum[node] -= 1                       # remove 1 every time a node is an indegree
            except:
                pass   

    for key,value in outdegrees_indegrees_sum.items():            
        if value > 0:
            more_efferents.append((key,value))
        if value < 0:
            more_afferents.append((key,value))
        else:
            pass
            
    if len(more_efferents)==1 and len(more_afferents) == 1  :              #Conditions for Eulirian Path. 2 unbalanced nodes
            if abs (more_efferents[0][1]) == abs (more_afferents[0][1]) :  #if the values are equal 
                return [more_efferents[0][0],more_afferents[0][0]]         #return the keys-nodes
    else :
        print('It is not possible to form a Eulirian Path')

In [ ]:
def Path_With_Joined_Edges (adjacency_dic, first_key, last_key):
    """Input 1: Dic with str as keys (a kmers' prefix [kmer[:-1]) and LIST with str as values (same kmers' suffix [kmer[1:]). 
                The key and its value(s) are adjacent nodes/vertices that together form a kmer
       Input 2: string or int
       Input 3: string or int
       Output : Dic with str as keys (a kmers' prefix [kmer[:-1]) and LIST with str as values (same kmers' suffix [kmer[1:]). 
                The key and its value(s) are adjacent nodes/vertices that together form a kmer
       
       In a dictionary from which a Eulirian Path can be formed, join the first and last unbalanced node to create a Eulirian cycle """
        
    if last_key in adjacency_dic.keys():
        adjacency_dic[last_key].append(first_key)
    else:
        adjacency_dic[last_key] = [first_key]
        
    return adjacency_dic